In [12]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [13]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

In [14]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,data_source2,other_projects,path,fold):
    df_train = pd.read_pickle(data_source1 + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    for _ in range(1):
        df = pd.read_csv(data_source1 + '/bellwether_cdom_1.csv')
        print(df)
        _cluster = {}
        for j in range(df.shape[0]):
            cluster_id = df.iloc[j,0]
            _bellwether = [df.iloc[j,7]]
            _cluster[cluster_id] = [_bellwether,list(df_train.iloc[cluster_tree[cluster_id].data_points].index)]       
        bell = birch_bellwether.bellwether(path,df)
        for _bell_cluster in _cluster:
            print(_cluster[_bell_cluster][0],_cluster[_bell_cluster][1])
            final_score = bell.bellwether(_cluster[_bell_cluster][0],_cluster[_bell_cluster][1])
            _results = {}
            goals = ['f1','g','recall','precision','pf']
            for goal in goals:
                for s_project in final_score.keys():
                    if s_project not in _results.keys():
                        _results[s_project] = {}
                    for d_projects in final_score[s_project].keys():
                        if goal == 'g':
                            _goal = 'g-score'
                        else:
                            _goal = goal
                        _results[s_project][d_projects] = np.median(final_score[s_project][d_projects][_goal])
                data_path = data_source2 + '/' + str(_bell_cluster)
                if not Path(data_path).is_dir():
                    os.makedirs(Path(data_path))
                _df_results = pd.DataFrame.from_dict(_results, orient='index')
                _df_results = _df_results.to_csv(data_path +'/bellwether_' + goal +'.csv')
                _df_results = pd.read_csv(data_path +'/bellwether_' + goal +'.csv')
        

In [15]:
for i in range(0,4):
    print(i)
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/2/fold_' + str(i)
    data_source2 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp2/0/cdom/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = path + '/'
    else:
        _dir = path + '\\'

    projects = [f for f in listdir(_dir) if isfile(join(_dir, f))]
    #print(projects)
    find_bellwether(data_source1,data_source2,projects,path,i)
    

0
   Unnamed: 0     f1      g  recall  precision     pf  cdom  \
0           3  0.590  0.450    0.49      0.470  0.460  54.0   
1           9  0.665  0.630    0.63      0.630  0.615   9.0   
2          20  0.670  0.530    0.56      0.550  0.540  15.0   
3          38  0.540  0.540    0.57      0.540  0.510  19.0   
4           1  0.990  0.990    0.99      0.990  0.990   0.0   
5           2  0.520  0.745    0.97      0.745  0.520   0.0   
6          37  0.740  0.740    0.74      0.740  0.720   1.0   

            bellwether  
0          glelite.csv  
1            yield.csv  
2  ivataopenportal.csv  
3         qtractor.csv  
4           benojt.csv  
5        emftriple.csv  
6       javagroups.csv  
['glelite.csv'] ['nutz.csv', 'jajuk.csv', 'ossim.csv', 'massiv.csv', 'v8.csv', 'ngl.csv', 'qmmp.csv', 'mesa3d.csv', 'rcp-company-uibindings.csv', 'vapor.csv', 'zscreen.csv', 'autoopencas.csv', 'mbse.csv', 'avisynth2.csv', 'lportal.csv', 'les-indemodables.csv', 'simplewebservices.csv', 'mplaye

['benojt.csv'] ['benojt.csv']
benojt.csv
['emftriple.csv'] ['emftriple.csv']
emftriple.csv
['javagroups.csv'] ['javagroups.csv', 'ivef-sdk.csv']
javagroups.csv
1
   Unnamed: 0     f1     g  recall  precision    pf  cdom         bellwether
0           2  0.565  0.57    0.58      0.565  0.52  19.0  oscarmcmaster.csv
1          23  0.700  0.59    0.56      0.560  0.59   6.0         jmoney.csv
2          31  0.540  0.52    0.53      0.495  0.44   3.0         irplus.csv
3          36  0.575  0.60    0.59      0.600  0.57  11.0        twostep.csv
4          49  0.575  0.50    0.54      0.530  0.51  13.0    openmalaria.csv
5           1  0.200  0.50    0.80      0.500  0.25   0.0         atunes.csv
['oscarmcmaster.csv'] ['encog-java.csv', 'h2database.csv', 'raven-monitoring.csv', 'jaql.csv', 'bochs.csv', 'fontforge.csv', 'jfreereport.csv', 'freelords.csv', 'super-tux.csv', 'capcode.csv', 'lmms.csv', 'linuxsh.csv', 'mcmc-jags.csv', 'scite-ru.csv', 'bl-toolkit.csv', 'nicepress.csv', 'sbml.csv',

['openmalaria.csv'] ['umber.csv', 'ivatamasks.csv', 'jaxen.csv', 'tcl.csv', 'cuberok.csv', 'ivataopenportal.csv', 'rodin-b-sharp.csv', 'gwanted.csv', 'exist.csv', 'wheat.csv', 'google-web-toolkit.csv', 'codepurge.csv', 'rockbox.csv', 'reactos-mirror.csv', 'qgo.csv', 'kmatplot.csv', 'nh3d.csv', 'taokgame.csv', 'mcore3d.csv', 'applet2app.csv', 'gbif-providertoolkit.csv', 'htmlunit.csv', 'qtwin.csv', 'unbvision.csv', 'wishmaster.csv', 'autat.csv', 'all-plasm.csv', 'acdk.csv', 'autowikibrowser.csv', 'uwom.csv', 'plearn.csv', 'nsis.csv', 'aztec.csv', 'papertoolkit.csv', 'qdvdauthor.csv', 'tango-cs.csv', 'thrust.csv', 'k3b.csv', 'mp-rechnungs-und-kundenverwaltung.csv', 'fityk.csv', 'rhex.csv', 'wgs-assembler.csv', 'etics.csv', 'pcgen.csv', 'aime.csv', 'lockss.csv', 'stuproa-cims.csv', 'personalaccess.csv', 'uclmda.csv', 'pydev.csv', 'elateportal.csv', 'awemud.csv', 'lsb.csv', 'rtb-team.csv', 'qgis.csv', 'upp-mirror.csv', 'open-beos.csv', 'firebreath.csv', 'maya-work-in-progress.csv', 'corner

['xcf.csv'] ['heekscad.csv', 'gamestone.csv', 'xcsoar.csv', 'igarden.csv', 'gbif-providertoolkit.csv', 'piccolo2d.csv', 'levent.csv', 'jstock.csv', 'pnotepad.csv', 'trinitycore.csv', 'geochat.csv', 'plearn.csv', 'awemud.csv', 'tzod.csv', 'psrchive.csv', 'liferea.csv', 'kwave.csv', 'firebreath.csv', 'qcad2.csv', 'customwars.csv', 'irplus.csv', 'wxcode.csv', 'xinity.csv', 'xmoto.csv', 'gss.csv', 'octave.csv', 'zildo.csv', 'customsagetv.csv', 'alleg.csv', 'ctrlr.csv', 'xfce.csv', 'glas.csv', 'xcf.csv', 'tango-cs.csv', 'olsrd.csv', 'silvertree.csv', 'freewrl.csv', 'adaptagrams.csv', 'go.csv', 'bibedt.csv', 'moving-pictures.csv', 'libmesh.csv', 'glelite.csv', 'fulguro.csv']
xcf.csv
['columba.csv'] ['encog-java.csv', 'h2database.csv', 'raven-monitoring.csv', 'jaql.csv', 'v8.csv', 'fluent-nhibernate.csv', 'elmo.csv', 'vapor.csv', 'nicepress.csv', 'etherboot.csv', 'gpac.csv', 'gebr.csv', 'libtorrent.csv', 'modellus.csv', 'ffigo.csv', 'mplayer-ce.csv', 'miranda.csv', 'jajuk.csv', 'autoopencas.c

['rectracker.csv'] ['openwbem.csv', 'nova.csv', 'crisscross.csv', 'mb-unit.csv', 'ivataopenportal.csv', 'elynx.csv', 'cascading.csv', 'aten.csv', 'xcmetadataservicestoolkit.csv', 'lite.csv', 'codepurge.csv', 'olex2.csv', 'stajistics.csv', 'jfreechart.csv', 'elvishrayrenderer.csv', 'uwom.csv', 'gisgraphy.csv', 'net-snmp.csv', 'powerfolder-.csv', 'hatari.csv', 'opde.csv', 'ltp.csv', 'chipster.csv', 'tcl.csv', 'decidr.csv', 'glas.csv', 'awemud.csv', 'tzod.csv', 'freebios.csv', 'k3b.csv', 'quarkplusplus.csv', 'powerfolder.csv', 'loki-lib.csv', 'x4x.csv', 'graphmak.csv', 'rectracker.csv', 'pdfedit.csv', 'wxlua.csv', 'jruby.csv', 'jahshakafx.csv', 'unladen-swallow.csv', 'plee-the-bear.csv', 'reaper-ecad.csv', 'nullc.csv', 'ns-3.csv', 'novembre.csv', 'vienna-add-in.csv', 'mptvseries.csv', 'sbml.csv', 'wow-qrsk.csv', 'mp-rechnungs-und-kundenverwaltung.csv', 'galaxium.csv', 'itext.csv', 'smartgwt.csv', 'fityk.csv', 'mecat.csv', 'kolmafia.csv', 'pixel-commandos.csv', 'quantlib.csv', 'smbopensoft